In [3]:
import os
import json
import re

In [4]:
def merge_details(backward_details):
    merged_list = []
    current_item = ""

    for i, item in enumerate(backward_details):
        if item.strip() and item.strip()[0].isdigit():
            j = 1
            while j < len(item.strip()) and item.strip()[j].isdigit():
                j += 1
            if j < len(item.strip()) and item.strip()[j] == "：":
                merged_list.append(current_item)
                current_item = item.strip()
            else:
                current_item += item.strip()
        else:
            current_item += item.strip()

    merged_list.append(current_item)
    merged_list.pop(0)
    return merged_list

In [11]:
def extract_tuples(string):
    tuples = []
    depth = 0
    start = None
    for i, char in enumerate(string):
        if char == "(" or char == '（' or char == '[':
            if depth == 0:
                start = i
            depth += 1
        elif char == ")" or char == '）' or char == ']':
            depth -= 1
            if depth == 0 and start is not None:
                tuples.append(string[start+1:i])
                start = None
    return tuples

def get_results(merged_list, input_list, rel_type_list, rel_extend_map, mode):
    input_list2rel = {}
    for idx, item in enumerate(input_list):
        input_list2rel[item] = rel_type_list[idx//10]

    rel_extend_index = 0
    cur_rel_extend_list = []
    A = []

    for idx, item in enumerate(merged_list):
        B = []
        cur_output_3 = item.split('Output:')
        cur_idx = cur_output_3[0].split('：')[0]
        cur_example = input_list[int(cur_idx)-1]
        cur_true_rel = input_list2rel[cur_example]

        if rel_extend_index == len(cur_rel_extend_list):
            rel_extend_index = 0

        cur_rel_extend_list = rel_extend_map[cur_true_rel]
        if not cur_rel_extend_list:
            continue
        try:
            cur_extend_rel = cur_rel_extend_list[rel_extend_index]
        except:
            print(idx, cur_true_rel)

        for output in cur_output_3[1:]:
            sep = ""
            if 'zh' in mode:
                if "生成关系列表：" in output:
                    sep = "生成关系列表："
                elif "生成关系列表:" in output:
                    sep = "生成关系列表:"
                elif "生成关系列表为：" in output:
                    sep = "生成关系列表为："
                elif "生成的关系列表为：" in output:
                    sep = "生成的关系列表为："
            elif 'en' in mode:
                if "Generated list of relations:" in output:
                    sep = "Generated list of relations:"
                elif "Generate a list of relations:" in output:
                    sep = "Generate a list of relations:"
                elif "List of Relations:" in output:
                    sep = "List of Relations:"
                elif "relation list:" in output:
                    sep = "relation list:"
                elif "Generate a list of relationships:" in output:
                    sep = "Generate a list of relationships:"
                elif "generate relationship list:" in output:
                    sep = "generate relationship list:"
                
            if sep:
                tmp = output.split(sep)[1]
                matches = extract_tuples(tmp)
                if matches:
                    for match in matches:
                        if 'zh' in mode:
                            triple = tuple(match.replace('，',',').strip().strip('（）').split(','))
                        elif 'en' in mode:
                            triple = tuple(match.strip().strip('()').split(','))
                        if len(triple) == 3: 
                            if 'zh' in mode:
                                if cur_extend_rel == triple[0].strip().strip('()').strip('""'):
                                    B.append((int(cur_idx)-1, cur_extend_rel, triple[1].strip().strip('()'), triple[2].strip().strip('()')))
                                elif cur_extend_rel == triple[1].strip().strip('()').strip('""'):
                                    B.append((int(cur_idx)-1, cur_extend_rel, triple[0].strip().strip('()'), triple[2].strip().strip('()')))
                                else:
                                    B.append((int(cur_idx)-1, cur_extend_rel, '', ''))
                            elif 'en' in mode:
                                if sep != "subject-object pair:" and sep!="Subject-object pair:":
                                    B.append((int(cur_idx)-1, cur_extend_rel, triple[0].strip().strip('()'), triple[2].strip().strip('()')))
                                else:
                                    B.append((int(cur_idx)-1, cur_extend_rel, '', ''))
                        elif len(triple) == 2:
                            if sep == "subject-object pair:" or sep == "Subject-object pair:":
                                B.append((int(cur_idx)-1, cur_extend_rel, triple[0].strip().strip('()'), triple[1].strip().strip('()')))
                            else:
                                B.append((int(cur_idx)-1, cur_extend_rel, '', ''))
                        else:
                            B.append((int(cur_idx)-1, cur_extend_rel, '', ''))

                else:
                    B.append((int(cur_idx)-1, cur_extend_rel, '', ''))
                
            else:
                B.append((int(cur_idx)-1, cur_extend_rel, '', ''))
                continue
        if B:
            A.append(B)    

        rel_extend_index += 1

    A = [list(set(item)) for item in A]
    
    with open(f'./results/backward_{mode}.txt','w',encoding='utf-8') as f:
        for item in A:
            f.write(str(item)+'\n')


def get_gold_results(merged_list, input_list, rel_type_list, mode):
    input_list2rel = {}
    for idx, item in enumerate(input_list):
        input_list2rel[item] = rel_type_list[idx//10]

    A = []

    for idx, item in enumerate(merged_list):
        B = []
        cur_output_3 = item.split('Output:')
        cur_idx = cur_output_3[0].split('：')[0]
        cur_example = input_list[int(cur_idx)-1]
        cur_true_rel = input_list2rel[cur_example]


        for output in cur_output_3[1:]:
            sep = ""
            if 'zh' in mode:
                if "生成关系列表：" in output:
                    sep = "生成关系列表："
                elif "生成关系列表:" in output:
                    sep = "生成关系列表:"
                elif "生成关系列表为：" in output:
                    sep = "生成关系列表为："
                elif "生成的关系列表为：" in output:
                    sep = "生成的关系列表为："
            elif 'en' in mode:
                if "Generated list of relations:" in output:
                    sep = "Generated list of relations:"
                elif "Generate a list of relations:" in output:
                    sep = "Generate a list of relations:"
                elif "List of Relations:" in output:
                    sep = "List of Relations:"
                elif "relation list:" in output:
                    sep = "relation list:"
                elif "Generate a list of relationships:" in output:
                    sep = "Generate a list of relationships:"
                elif "generate relationship list:" in output:
                    sep = "generate relationship list:"
                
            if sep:
                tmp = output.split(sep)[1]
                matches = extract_tuples(tmp)
                if matches:
                    for match in matches:
                        if 'zh' in mode:
                            triple = tuple(match.strip().replace('，',',').strip('（）').split(','))
                        elif 'en' in mode:
                            triple = tuple(match.strip().strip('()').split(','))
                        if len(triple) == 3 and sep != "subject-object pair: ":
                            B.append((int(cur_idx)-1, cur_true_rel, triple[0].strip().strip('()'), triple[2].strip().strip('()')))
                        elif len(triple) == 2 and sep == "subject-object pair: ":
                            B.append((int(cur_idx)-1, cur_true_rel, triple[0].strip().strip('()'), triple[1].strip().strip('()')))
                        else:
                            B.append((int(cur_idx)-1, cur_true_rel, '', ''))
                else:
                    B.append((int(cur_idx)-1, cur_true_rel, '', ''))
                
            else:
                B.append((int(cur_idx)-1, cur_true_rel, '', ''))
                continue
        if B:
            A.append(B)    

    A = [list(set(item)) for item in A]
    
    with open(f'./results/backward_gold_{mode}.txt','w',encoding='utf-8') as f:
        for item in A:
            f.write(str(item)+'\n')

CMeIE

In [6]:
with open('./results/details_zh.txt','r',encoding='utf-8') as f:
    backward_details = f.readlines()

with open('./results/details_gold_zh.txt','r',encoding='utf-8') as f:
    backward_gold_details = f.readlines()

with open('./data/cmeie/input_list.txt','r',encoding='utf-8') as f:
    input_list = [item.strip() for item in f.readlines()]

with open('./data/cmeie/final_rel_extend_map_zh.json','r',encoding='utf-8') as f:
    rel_extend_map = json.load(f)

rel_type_list = list(rel_extend_map.keys())

# merged_list = merge_details(backward_details)
# get_results(merged_list, input_list, rel_type_list, rel_extend_map, 'zh')

merged_list = merge_details(backward_gold_details)
get_gold_results(merged_list, input_list, rel_type_list, 'zh')

SCIERC

In [ ]:
with open('./results/details_en.txt','r',encoding='utf-8') as f:
    backward_details = f.readlines()

with open('./results/details_gold_en.txt','r',encoding='utf-8') as f:
    backward_gold_details = f.readlines()

with open('./data/scierc/input_list.txt','r',encoding='utf-8') as f:
    input_list = [item.strip() for item in f.readlines()]

with open('./data/scierc/final_rel_extend_map_en.json','r',encoding='utf-8') as f:
    rel_extend_map = json.load(f)

rel_type_list = list(rel_extend_map.keys())

merged_list = merge_details(backward_details)
get_results(merged_list, input_list, rel_type_list, rel_extend_map, 'en')
# merged_list = merge_details(backward_gold_details)
# get_gold_results(merged_list, input_list, rel_type_list, 'en')

Alpaca-CMeIE

In [21]:
with open('./results/details_alpaca_33B_zh.txt','r',encoding='utf-8') as f:
    backward_details = f.readlines()

with open('./results/details_gold_alpaca_33B_zh.txt','r',encoding='utf-8') as f:
    backward_gold_details = f.readlines()

with open('./data/cmeie/input_list.txt','r',encoding='utf-8') as f:
    input_list = [item.strip() for item in f.readlines()]

with open('./data/cmeie/final_rel_extend_map_alpaca_33B_zh.json','r',encoding='utf-8') as f:
    rel_extend_map = json.load(f)

rel_type_list = list(rel_extend_map.keys())

# merged_list = merge_details(backward_details)
# get_results(merged_list, input_list, rel_type_list, rel_extend_map, 'alpaca_33B_zh')

merged_list = merge_details(backward_gold_details)
get_gold_results(merged_list, input_list, rel_type_list, 'alpaca_33B_zh')

Alpaca-SCIERC

In [ ]:
with open('./results/details_alpaca_33B_en.txt','r',encoding='utf-8') as f:
    backward_details = f.readlines()

with open('./results/details_gold_alpaca_33B_en.txt','r',encoding='utf-8') as f:
    backward_gold_details = f.readlines()

with open('./data/scierc/input_list.txt','r',encoding='utf-8') as f:
    input_list = [item.strip() for item in f.readlines()]

with open('./data/scierc/final_rel_extend_map_alpaca_33B_en.json','r',encoding='utf-8') as f:
    rel_extend_map = json.load(f)

rel_type_list = list(rel_extend_map.keys())

merged_list = merge_details(backward_details)
get_results(merged_list, input_list, rel_type_list, rel_extend_map, 'alpaca_33B_en')

# merged_list = merge_details(backward_gold_details)
# get_gold_results(merged_list, input_list, rel_type_list, 'alpaca_33B_en')

Llama2-SCIERC

In [23]:
with open('./results/details_llama2_70B_en.txt','r',encoding='utf-8') as f:
    backward_details = f.readlines()

with open('./results/details_gold_llama2_70B_en.txt','r',encoding='utf-8') as f:
    backward_gold_details = f.readlines()

with open('./data/scierc/input_list.txt','r',encoding='utf-8') as f:
    input_list = [item.strip() for item in f.readlines()]

with open('./data/scierc/final_rel_extend_map_llama2_70B_en.json','r',encoding='utf-8') as f:
    rel_extend_map = json.load(f)

rel_type_list = list(rel_extend_map.keys())

# merged_list = merge_details(backward_details)
# get_results(merged_list, input_list, rel_type_list, rel_extend_map, 'llama2_70B_en')

merged_list = merge_details(backward_gold_details)
get_gold_results(merged_list, input_list, rel_type_list, 'llama2_70B_en')

ChatGLM-CMeIE

In [24]:
with open('./results/details_chatglm_6B_zh.txt','r',encoding='utf-8') as f:
    backward_details = f.readlines()

with open('./results/details_gold_chatglm_6B_zh.txt','r',encoding='utf-8') as f:
    backward_gold_details = f.readlines()

with open('./data/cmeie/input_list.txt','r',encoding='utf-8') as f:
    input_list = [item.strip() for item in f.readlines()]

with open('./data/cmeie/final_rel_extend_map_chatglm_6B_zh.json','r',encoding='utf-8') as f:
    rel_extend_map = json.load(f)

rel_type_list = list(rel_extend_map.keys())

# merged_list = merge_details(backward_details)
# get_results(merged_list, input_list, rel_type_list, rel_extend_map, 'chatglm_6B_zh')

merged_list = merge_details(backward_gold_details)
get_gold_results(merged_list, input_list, rel_type_list, 'chatglm_6B_zh')

ChatGLM-SCIERC

In [11]:
with open('./results/details_chatglm_6B_en.txt','r',encoding='utf-8') as f:
    backward_details = f.readlines()

with open('./results/details_gold_chatglm_6B_en.txt','r',encoding='utf-8') as f:
    backward_gold_details = f.readlines()

with open('./data/scierc/input_list.txt','r',encoding='utf-8') as f:
    input_list = [item.strip() for item in f.readlines()]

with open('./data/scierc/final_rel_extend_map_chatglm_6B_en.json','r',encoding='utf-8') as f:
    rel_extend_map = json.load(f)

rel_type_list = list(rel_extend_map.keys())

merged_list = merge_details(backward_details)
get_results(merged_list, input_list, rel_type_list, rel_extend_map, 'chatglm_6B_en')

# merged_list = merge_details(backward_gold_details)
# get_gold_results(merged_list, input_list, rel_type_list, 'chatglm_6B_en')

GPT4-CMeIE

In [7]:
with open('./results/details_gpt4_zh.txt','r',encoding='utf-8') as f:
    backward_details = f.readlines()

with open('./data/cmeie/input_list.txt','r',encoding='utf-8') as f:
    input_list = [item.strip() for item in f.readlines()]

with open('./data/cmeie/final_rel_extend_map_gpt4_zh.json','r',encoding='utf-8') as f:
    rel_extend_map = json.load(f)

rel_type_list = list(rel_extend_map.keys())

merged_list = merge_details(backward_details)
get_results(merged_list, input_list, rel_type_list, rel_extend_map, 'gpt4_zh')

GPT4-SCIERC

In [8]:
with open('./results/details_gpt4_en.txt','r',encoding='utf-8') as f:
    backward_details = f.readlines()

with open('./data/scierc/input_list.txt','r',encoding='utf-8') as f:
    input_list = [item.strip() for item in f.readlines()]

with open('./data/scierc/final_rel_extend_map_gpt4_en.json','r',encoding='utf-8') as f:
    rel_extend_map = json.load(f)

rel_type_list = list(rel_extend_map.keys())

merged_list = merge_details(backward_details)
get_results(merged_list, input_list, rel_type_list, rel_extend_map, 'gpt4_en')

Baichuan2-CMeIE

In [5]:
with open('./results/details_baichuan2_13B_zh.txt','r',encoding='utf-8') as f:
    backward_details = f.readlines()

with open('./results/details_gold_baichuan2_13B_zh.txt','r',encoding='utf-8') as f:
    backward_gold_details = f.readlines()

with open('./data/cmeie/input_list.txt','r',encoding='utf-8') as f:
    input_list = [item.strip() for item in f.readlines()]

with open('./data/cmeie/final_rel_extend_map_baichuan2_13B_zh.json','r',encoding='utf-8') as f:
    rel_extend_map = json.load(f)

rel_type_list = list(rel_extend_map.keys())

merged_list = merge_details(backward_details)
get_results(merged_list, input_list, rel_type_list, rel_extend_map, 'baichuan2_13B_zh')

# merged_list = merge_details(backward_gold_details)
# get_gold_results(merged_list, input_list, rel_type_list, 'baichuan2_13B_zh')

Baichuan2-SCIERC

In [12]:
with open('./results/details_baichuan2_13B_en.txt','r',encoding='utf-8') as f:
    backward_details = f.readlines()

with open('./results/details_gold_baichuan2_13B_en.txt','r',encoding='utf-8') as f:
    backward_gold_details = f.readlines()

with open('./data/scierc/input_list.txt','r',encoding='utf-8') as f:
    input_list = [item.strip() for item in f.readlines()]

with open('./data/scierc/final_rel_extend_map_baichuan2_13B_en.json','r',encoding='utf-8') as f:
    rel_extend_map = json.load(f)

rel_type_list = list(rel_extend_map.keys())

# merged_list = merge_details(backward_details)
# get_results(merged_list, input_list, rel_type_list, rel_extend_map, 'baichuan2_13B_en')

merged_list = merge_details(backward_gold_details)
get_gold_results(merged_list, input_list, rel_type_list, 'baichuan2_13B_en')